In [2]:
#import modules

import numpy as np
import os
import matplotlib.pyplot as plt
import pickle

from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from keras.utils import Sequence
from keras.models import load_model
from tensorflow.distribute import MirroredStrategy

#initialise random generator
rng = np.random.default_rng()

In [11]:
#Define FelixDataflow classes and functions.

class FelixSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size, file_type):
        """Here self.x is a list of paths to file_type files. self.y is a
        corresponding list of labels."""
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.file_type = file_type

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return arrs_from_paths(batch_x, self.file_type), to_categorical(np.array(batch_y),10)

def gen_paths_labels(base_path):
    """A generator to yield (data-paths, corresponding labels) tuples for each
    segment of data (typically training, validation, and testing)."""
    for segment in sorted(os.listdir(base_path)):
        segment_path = os.path.join(base_path, segment)
        segment_paths = []
        segment_labels = []
        for label in os.listdir(segment_path):
            label_path = os.path.join(segment_path, label)
            for crystal in os.listdir(label_path):
                segment_paths.append(os.path.join(label_path, crystal))
                segment_labels.append(label)
        indexes = np.arange(len(segment_labels))
        rng.shuffle(indexes)
        yield [np.array(segment_paths)[indexes], np.array(list(map(int,segment_labels)))[indexes]]

def arrs_from_paths(paths, file_type):
    if file_type == "txt":
        return np.array([np.loadtxt(file_name) for file_name in paths])
    elif file_type == "npy":
        return np.array([np.load(file_name)[[0],:,:] for file_name in paths])

def felix_fit_new(model, batch_size, epochs, workers, AllPaths, file_type, patience):
    #AllPaths = [[TrainingPaths, TrainingThickness], [], []]
    """A fit function to allow validation and test data to be supplied via a
    generator."""
    
    best_val_acc = -np.inf
    
    best_model = None
    
    x = np.arange(0, epochs)
    
    train_seq = FelixSequence(AllPaths[0][0], AllPaths[0][1], batch_size, file_type)
    val_seq = FelixSequence(AllPaths[1][0], AllPaths[1][1], batch_size, file_type)
    test_seq = FelixSequence(AllPaths[2][0], AllPaths[2][1], batch_size, file_type)
    for epoch in range(epochs):
        print("-------------------------------------------------------------------------")
        print("Epoch", epoch+1, "/", epochs, ": ")
        print("Training: ")
        train_hist = model.fit(x = train_seq, epochs = epoch+1, workers = workers, initial_epoch = epoch, shuffle=True)

        print("Validation: ")
        val_hist = model.evaluate(x = val_seq, workers = workers)       
        epoch_acc = val_hist[1]
        if(epoch_acc > best_val_acc):
            best_model = model
            #model.save(NewPath+ModelName)
            print("The model improved from: ",best_val_acc, "to: ", epoch_acc)
            best_val_acc = epoch_acc
            patience_i = 0
        else:
            patience_i+=1
            print("The model did not improve, patience_i = ", patience_i)

        print("Epoch acc: ", epoch_acc)
        #val_hist[0][epoch] = avg_recon_loss
        if(patience_i > patience):
            print("Early Stopping, the model did not improve from: ", best_val_acc)
            break

    print("-------------------------------------------------------------------------")
    print("Testing: ")
    tst_hist = best_model.evaluate(test_seq, workers = workers)
    
    return tst_hist[1], best_model

    

In [4]:
def MakeThicknessList(ListPaths):
    Thickness = []
    for i in ListPaths:
        Thickness.append(int(i.split("/")[-1].split(".")[0]))
    Thickness = np.array(Thickness)
    return(Thickness)

def OpenTxt(Path):
    with open(Path) as textFile:
        lines = [line.split() for line in textFile]
    List = []
    for i in lines:
        List.append(i[0])
    return(List)


DataPath = "//home/ug-ml/felix-ML/classification/Classification000/DataPaths/"

TrainPathAll = OpenTxt(DataPath + "Train_0p1.txt")
ValPathAll = OpenTxt(DataPath + "Validation_0p1.txt")
TestPathAll = OpenTxt(DataPath + "Test_0p1.txt")

In [5]:
LimTrain = int(len(TrainPathAll) * 0.1)
LimVal = int(len(ValPathAll) * 0.1)
LimTest = int(len(TestPathAll) * 0.1)

TrainPath = TrainPathAll[0:LimTrain]
ValPath = ValPathAll[0:LimVal]
TestPath = TestPathAll[0:LimTest]

TrainThickness = MakeThicknessList(TrainPath)[0:LimTrain]
ValThickness = MakeThicknessList(ValPath)[0:LimVal]
TestThickness = MakeThicknessList(TestPath)[0:LimTest]




AllPaths = [[TrainPath,TrainThickness],[ValPath,ValThickness],[TestPath,TestThickness]]

In [6]:
#All paths

SaveDataPath = "/home/ug-ml/Documents/GitHub_BigFiles/SaveFolder" #Base directory of place you store information of models
SaveFolderName = "/Classifer_1" #Will create a folder and put in information about the outcome / inputs
ModelName = "/Model.hdf5"

NewPath = SaveDataPath + SaveFolderName
#Many variables

#Model Variables
input_shape = (1, 128, 128)

#Hyper parameters
learning_rate = 0.0005
l2_regularizer = 0.0001
loss = 'categorical_crossentropy'
optimizer = "RMSprop" #Not a variable ONLY used for a note
batch_size = 32
epochs = 50
ShuffleTrainData = True

#Call back variables
TrainingPatience = 5
CheckPointMonitor = 'val_acc'
EarlyStopMonitor = 'val_acc'

#CPU variables
CPUworkers = 16


#List the name of the variables you want to save in a file
VariableListName = ["input_shape", 
                   "learning_rate", "l2_regularizer", "loss", "optimizer", "batch_size", "epochs", "ShuffleTrainData",
                   "TrainingPatience", "CheckPointMonitor", "EarlyStopMonitor",
                   "CPUworkers"]

#List the variables in the same order as VariableListName
VariableListValues = [input_shape, 
                   learning_rate, l2_regularizer, loss, optimizer, batch_size, epochs, ShuffleTrainData,
                   TrainingPatience, CheckPointMonitor, EarlyStopMonitor,
                   CPUworkers]

In [7]:
def build_model(learning_rate, l2_regularizer, dense_layer_size, kernal_size):
    
    dense_layer_size = int(dense_layer_size)
    kernal_size = int(kernal_size)
    strategy = MirroredStrategy() #Allows multiple GPUs

    with strategy.scope():
        model = models.Sequential()
        model.add(layers.Conv2D(128, (kernal_size, kernal_size),
                                         activation='relu',
                                         data_format='channels_first',
                                         input_shape= input_shape))
        model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
        model.add(layers.Conv2D(128, (kernal_size, kernal_size),
                                         data_format='channels_first',
                                         activation='relu'))
        model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
        model.add(layers.Conv2D(128, (kernal_size, kernal_size),
                                         data_format='channels_first',
                                         activation='relu'))
        model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
        model.add(layers.Flatten())
        model.add(layers.Dropout(0.25))
        model.add(layers.Dense(dense_layer_size, activation='relu',
                               kernel_regularizer = l2(l2_regularizer)))

        model.add(layers.Dense(10, activation='softmax',
                               kernel_regularizer = l2(l2_regularizer)))

        model.compile(loss = loss,
                      optimizer = optimizers.RMSprop(learning_rate = learning_rate),
                      metrics=['acc'])
    return model

In [ ]:
parameters = {"learning rate": np.array([0.0005, 0.0001, 0.001, 0.002, 0.005]),
             "l2_regulariser": np.array([0.00005, 0.0001, 0.0005, 0.001, 0.002]),
             "batch_size": np.array([8, 16, 32, 64, 128]),
             "dense_layer_size":np.array([32, 64, 128, 256, 512]),
             "kernal_size": np.array([2, 4, 6, 8, 10])}
    
z = np.array([0,1,2,1,1])
prev_searched = np.array([[0,1,2,1,1]])

def neighbours(point, dircs):
    ns = dircs+point
    #print(dircs, point, prev_searched)
    return np.array([i for i in ns if (0<=i).all() and (i<5).all() and not (i == prev_searched).all(axis=1).any()]).astype(int)

def parameter_search(parameters, z, prev_searched):
    
    num_params = len(parameters)
    dircs = np.zeros(shape = (2*num_params, num_params))
    for i, dirc in enumerate(dircs):
        if i < num_params:
            dirc[i] = 1
        else:
            dirc[i-num_params] = -1
    
    best_params = {}
    for key in parameters:
        best_params[key] = np.nan

    best_metrics = {"test_loss": np.inf, "test_acc": -np.inf}

    best_hist = {"train_acc": np.zeros(shape = epochs),
                "train_loss": np.zeros(shape=epochs),
                "val_acc": np.zeros(shape = epochs),
                "val_loss": np.zeros(shape=epochs)}
    
    converged = False
        
    while not converged:
        print("ITERATING OVER:")
        neighs = neighbours(z, dircs)
        num_neighs = neighs.shape[0]
        print(neighs, num_neighs)

        if neighs.size == 0:
            print("No new neighbours available. Saving best model and parameter set so far.")
            best_model.save(NewPath+ModelName)
            converged = True
            break

        step_params = np.array([parameters[key][neighs[:,j]] for j, key in enumerate(parameters)]).T
        #print(step_params)
                

        converged = True

        for i, param_set in enumerate(step_params):
            print("Parameter set ", i+1," of ", num_neighs,"+++++++++++++++++++++++++++++++++++++++++++++++++++.")

            model = build_model(param_set[0], param_set[1], param_set[3], param_set[4])

            test_acc, best_model = felix_fit_new(model, param_set[2].astype(int), epochs, CPUworkers, AllPaths, "npy", TrainingPatience)

            prev_searched = np.append(prev_searched, neighs[i].reshape(1,num_params), axis=0)

            if test_acc > best_metrics["test_acc"]:
                
                best_metrics["test_acc"] = test_acc
                
                for i, key in enumerate(best_params):
                    best_params[key] = param_set[i]

                z = neighs[i]
                
                best_model.save(NewPath+ModelName)
                converged = False
        print("best params set:" )
        print(best_params)
                
parameter_search(parameters, z, prev_searched)

ITERATING OVER:
[[1 1 2 1 1]
 [0 2 2 1 1]
 [0 1 3 1 1]
 [0 1 2 2 1]
 [0 1 2 1 2]
 [0 0 2 1 1]
 [0 1 1 1 1]
 [0 1 2 0 1]
 [0 1 2 1 0]] 9
Parameter set  1  of  9 +++++++++++++++++++++++++++++++++++++++++++++++++++.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
-------------------------------------------------------------------------
Epoch 1 / 50 : 
Training: 
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 10 all-reduces with algorithm = nccl, num_packs = 1
177/177 [==============================] - 6s 34ms/step - loss: 2.0571 - acc: 0.2895
Validation: 
21/21 [==============================] - 1s 27ms/step - loss: 2.6650 - acc: 0.3383
The model improved from:  -inf to:  0.33834585547447205
Epoch acc:  0.33834585547447205
-------------------------------------------------------------------------
Epoch 2 / 50 : 
Traini

Epoch 21/21
177/177 [==============================] - 6s 35ms/step - loss: 0.6630 - acc: 0.7911
Validation: 
21/21 [==============================] - 1s 25ms/step - loss: 2.4055 - acc: 0.6015
The model did not improve, patience_i =  2
Epoch acc:  0.6015037298202515
-------------------------------------------------------------------------
Epoch 22 / 50 : 
Training: 
Epoch 22/22
177/177 [==============================] - 6s 37ms/step - loss: 0.5854 - acc: 0.8045
Validation: 
21/21 [==============================] - 1s 31ms/step - loss: 2.2233 - acc: 0.6165
The model improved from:  0.6030074954032898 to:  0.61654132604599
Epoch acc:  0.61654132604599
-------------------------------------------------------------------------
Epoch 23 / 50 : 
Training: 
Epoch 23/23
177/177 [==============================] - 6s 35ms/step - loss: 0.5704 - acc: 0.8125
Validation: 
21/21 [==============================] - 1s 27ms/step - loss: 2.8269 - acc: 0.6060
The model did not improve, patience_i =  1
Epoc

21/21 [==============================] - 1s 24ms/step - loss: 4.1462 - acc: 0.6707
The model did not improve, patience_i =  1
Epoch acc:  0.6706767082214355
-------------------------------------------------------------------------
Epoch 43 / 50 : 
Training: 
Epoch 43/43
177/177 [==============================] - 7s 38ms/step - loss: 0.2549 - acc: 0.9226
Validation: 
21/21 [==============================] - 1s 26ms/step - loss: 3.5333 - acc: 0.6782
The model did not improve, patience_i =  2
Epoch acc:  0.6781954765319824
-------------------------------------------------------------------------
Epoch 44 / 50 : 
Training: 
Epoch 44/44
177/177 [==============================] - 7s 39ms/step - loss: 0.2295 - acc: 0.9284
Validation: 
21/21 [==============================] - 1s 28ms/step - loss: 3.8083 - acc: 0.6722
The model did not improve, patience_i =  3
Epoch acc:  0.6721804738044739
-------------------------------------------------------------------------
Epoch 45 / 50 : 
Training: 
Epo

177/177 [==============================] - 6s 34ms/step - loss: 0.2999 - acc: 0.9491
Validation: 
21/21 [==============================] - 0s 22ms/step - loss: 2.4162 - acc: 0.6962
The model did not improve, patience_i =  3
Epoch acc:  0.6962406039237976
-------------------------------------------------------------------------
Epoch 16 / 50 : 
Training: 
Epoch 16/16
177/177 [==============================] - 6s 35ms/step - loss: 0.2768 - acc: 0.9523
Validation: 
21/21 [==============================] - 0s 17ms/step - loss: 3.0457 - acc: 0.7143
The model improved from:  0.7067669034004211 to:  0.7142857313156128
Epoch acc:  0.7142857313156128
-------------------------------------------------------------------------
Epoch 17 / 50 : 
Training: 
Epoch 17/17
177/177 [==============================] - 7s 42ms/step - loss: 0.2550 - acc: 0.9593: 0s - loss: 0.2532 - acc: 0.95
Validation: 
21/21 [==============================] - 1s 27ms/step - loss: 3.0710 - acc: 0.7188
The model improved from:

In [110]:
prev_searched = np.array([[0,1,2]])

learning_rate = np.array([0.0005, 0.0001, 0.001, 0.002, 0.005])
l2_regularizer = np.array([0.00005, 0.0001, 0.0005, 0.001, 0.002])
batch_size = np.array([8, 16, 32, 64, 128])

def neighbours(point):
    dircs = np.array([[0,0,1],[0,1,0],[1,0,0],[0,0,-1],[0,-1,0],[-1,0,0]])
    ns = dircs+point
    return np.array([i for i in ns if (0<=i).all() and (i<5).all() and not (i == prev_searched).all(axis=1).any()])


z = np.array([0,1,2])
converged = False
best_test_loss = np.inf
best_lr = np.nan
best_l2_r = np.nan
best_bs = np.nan

while not converged:
    print("ITERATING OVER:")
    neighs = neighbours(z)
    print(neighs)
    #print(neighs)
    if neighs.size == 0:
        print("No new neighbours available. Saving best model and parameter set so far.")
        np.save(NewPath+"/parameter_search.npy", np.array(best_lr,best_l2_r,best_bs))
        best_model.save(NewPath+ModelName)
        converged = True
        break
        

    lr = learning_rate[neighs[:,0]]
    l2_r = l2_regularizer[neighs[:,1]]
    bs = batch_size[neighs[:,2]]

    step_params = np.array([lr, l2_r, bs]).T
    
    converged = True
    
    for i, param_set in enumerate(step_params):
        
        model = build_model(param_set[0], param_set[1])
        
        test_loss, best_model = felix_fit_new(model, param_set[2].astype(int), epochs, CPUworkers, AllPaths, "npy", TrainingPatience)
        
        prev_searched = np.append(prev_searched, neighs[i].reshape(1,3), axis=0)
        
        if test_loss < best_test_loss:
            best_test_loss = test_loss
            best_lr = param_set[0]
            best_l2_r = param_set[1]
            best_bs = param_set[2]
            
            z = neighs[i]
            
            np.save(NewPath+"/parameter_search.npy",param_set)
            best_model.save(NewPath+ModelName)
            converged = False
            
    

ITERATING OVER:
[[0 1 3]
 [0 2 2]
 [1 1 2]
 [0 1 1]
 [0 0 2]]
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
-------------------------------------------------------------------------
Epoch 1 / 1 : 
Training: 
89/89 [==============================] - 7s 83ms/step - loss: 2.0421 - acc: 0.2734
Validation: 
11/11 [==============================] - 0s 29ms/step - loss: 2.5310 - acc: 0.3549
The model improved from:  inf to:  2.5309948921203613
Epoch loss:  2.5309948921203613
-------------------------------------------------------------------------
Testing: 
6/6 [==============================] - 0s 20ms/step - loss: 1.9448 - acc: 0.3724
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
-------------------------------------------------------------------------
Epoch 1 / 1 : 
Training: 
177/177 [=

In [112]:
print(best_lr,best_l2_r,best_bs)
print(best_test_loss)

0.0005 0.0005 32.0
1.822420358657837


In [38]:
test_seq = FelixSequence(AllPaths[2][0], AllPaths[2][1], best_bs.astype(int), "npy")
tst_hist = best_model.evaluate(test_seq, workers = 16)

11/11 [==============================] - 0s 10ms/step - loss: 1.6834 - acc: 0.6216


In [ ]:
#Build model
strategy = MirroredStrategy() #Allows multiple GPUs

with strategy.scope():
    model = models.Sequential()
    model.add(layers.Conv2D(128, (4, 4),
                                     activation='relu',
                                     data_format='channels_first',
                                     input_shape= input_shape))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Conv2D(128, (4, 4),
                                     data_format='channels_first',
                                     activation='relu'))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Conv2D(128, (4, 4),
                                     data_format='channels_first',
                                     activation='relu'))
    model.add(layers.MaxPooling2D((2, 2), data_format='channels_first'))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(128, activation='relu',
                           kernel_regularizer = l2(l2_regularizer)))
    
    model.add(layers.Dense(10, activation='softmax',
                           kernel_regularizer = l2(l2_regularizer)))

    model.compile(loss = loss,
                  optimizer = optimizers.RMSprop(learning_rate = learning_rate),
                  metrics=['acc'])
    
#Make folder to put model and history information
try:
    os.mkdir(NewPath)
except:
    print("Folder failed to be created, it may already exist")
    
File1  = open(NewPath +"/Parameters.txt", "w+")
if(len(VariableListName) == len(VariableListValues)):
    for i in range(0, len(VariableListName)):
        File1.write(VariableListName[i] + " " + str(VariableListValues[i]) + "\n")
    File1.close()
else:
    print("VariableListName and VariableListValues do not match up, so file can not be saved")

In [ ]:
training_history = np.zeros(shape=(2,epochs))
validation_history = np.zeros(shape=(2,epochs))
test_history = [0,0]
#print(model.metrics_names)
felix_fit_new(model, batch_size, epochs, CPUworkers, AllPaths, "npy",TrainingPatience)